In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np

# Process

## subset: 2~5 mutations only

In [26]:
prediction = pd.read_table('/home/v-tianbopeng/project/muformer-demo/assets/tables/predictions/aav_less5.tsv')

In [34]:
# subset = prediction.loc[prediction.pos.apply(lambda x: x.count(',')<=5), :]
subset = prediction.loc[prediction.pos.apply(lambda x: x.count(',')==1), :].copy()

In [11]:
subset.to_csv('/home/v-tianbopeng/project/muformer-demo/assets/predictions/aav_less5.tsv', sep='\t', index=False)

## data for surface plot

**load**

In [4]:
prediction = pd.read_table('/home/v-tianbopeng/project/muformer-demo/assets/tables/predictions/tem.tsv')

**subset**

In [5]:
# subset = prediction.loc[prediction.pos.apply(lambda x: x.count(',')<=5), :]
subset = prediction.loc[prediction.pos.apply(lambda x: x.count(',')==1), :].copy()

In [6]:
subset.loc[:, 'x'] = subset.pos.apply(lambda x: int(x.split(',')[0]))
subset.loc[:, 'y'] = subset.pos.apply(lambda x: int(x.split(',')[1]))

**pivot**

In [7]:
mean_score = subset.groupby(['x', 'y']).score.mean().reset_index().rename(columns={'score': 'mean_score'})
mean_pred = subset.groupby(['x', 'y']).prediction.mean().reset_index().rename(columns={'prediction': 'mean_prediction'})

In [8]:
pivot_score = mean_score.pivot('x', 'y', 'mean_score')
pivot_pred = mean_pred.pivot('x', 'y', 'mean_prediction')

In [60]:
pivot_score.loc[588, :] = np.nan
pivot_score.loc[:, 561] = np.nan
pivot_score = pivot_score.loc[:, pivot_score.columns.sort_values()]
pivot_pred.loc[588, :] = np.nan
pivot_pred.loc[:, 561] = np.nan
pivot_pred = pivot_pred.loc[:, pivot_pred.columns.sort_values()]

**add reverse**

In [64]:
complete_score = pivot_score.fillna(0) + pivot_score.T.fillna(0)
complete_pred = pivot_pred.fillna(0) + pivot_pred.T.fillna(0)

In [67]:
complete_pred.replace(0, np.nan, inplace=True)
complete_score.replace(0, np.nan, inplace=True)

**save**

In [68]:
complete_score.to_csv('/home/v-tianbopeng/project/muformer-demo/assets/tables/plotdata/aav_less5_score.tsv', sep='\t')
complete_pred.to_csv('/home/v-tianbopeng/project/muformer-demo/assets/tables/plotdata/aav_less5_pred.tsv', sep='\t')

## possible mutations

In [26]:
prediction = pd.read_table('/home/v-tianbopeng/project/muformer-demo/assets/tables/predictions/aav_less5.tsv')

In [25]:
prediction.mutant.str.split(',', expand=True).stack().reset_index(drop=True).drop_duplicates().sort_values().to_frame('mutation').to_csv('/home/v-tianbopeng/project/muformer-demo/assets/tables/mutations/tem_less5_mutations.tsv', sep='\t', index=False)